# ДЗ1
1. Загрузите набор данных lenta-ru-news с помощью библиотеки Corus для задачи классификации текстов по топикам (пригодятся атрибуты title, text, topic)
2. Подготовьте данные к обучению:  
    - Предобработайте данные: реализуйте оптимальную, на ваш взгляд, предобработку текстов (нормализация, очистка, стемминг/лемматизация и т.п.) и таргета.
    - **hint**: для ускорения обработки  и обучения можно ограничиться не всем датасетом, а его репрезентативной частью, например, размера 100_000.
    - Кратко опишите пайплайн, на котором остановились, и почему.
    - Разделите датасет на обучающую, валидационную и тестовую выборки со стратификацией в пропорции 60/20/20. В качестве целевой переменной используйте атрибут `topic`
3. Замерьте базовое качество с любым dummy-бейзлайном
4. Обучите модель `sklearn.linear_model.LogisticRegression` с двумя вариантами векторизации:
  - `sklearn.feature_extraction.text.CountVectorizer`
  - `sklearn.feature_extraction.text.TfidfVectorizer`
5. Попробуйте улучшить качество, подобрав оптимальные гиперпараметры трансформаций и модели на кросс-валидации
6. Оцените качество лучшего пайплайна на отложенной выборке


In [1]:
# Подгружаем все необходимые библиотеки в рамках данного Дз.

import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from joblib import Parallel, delayed

import pandas as pd
from sklearn.model_selection import train_test_split

## 1. Загружаем датасет

In [7]:
## загружаем данные
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [8]:
df = pd.read_csv('lenta-ru-news.csv.gz', compression='gzip')
df.dropna(inplace=True)

In [82]:
df['full_text'] = df['title'] + " " + df['text']
df['topic'].value_counts()

topic
Россия               155078
Мир                  136679
Экономика             76433
Спорт                 57902
Культура              53536
Наука и техника       53136
Бывший СССР           51370
Интернет и СМИ        44433
Из жизни              27519
Дом                   21734
Силовые структуры     11223
Ценности               7581
Бизнес                 7375
Путешествия            6370
69-я параллель         1268
Крым                    666
Культпросвет            340
Легпром                 114
Библиотека               65
Оружие                    3
ЧМ-2014                   2
МедНовости                1
Сочи                      1
Name: count, dtype: int64

Наблюдаем несбалансированную выборку, поэтому избавимся от тех классов, которые имеют меньше 2000 объектов.

In [83]:
category_counts = df["topic"].value_counts()
valid_categories = category_counts[category_counts >= 2000].index
df_filtered = df[df["topic"].isin(valid_categories)].copy()
df_filtered.shape

(710369, 6)

In [84]:
df_w = df_filtered[['full_text','topic']]
df_w['topic'].value_counts()

topic
Россия               155078
Мир                  136679
Экономика             76433
Спорт                 57902
Культура              53536
Наука и техника       53136
Бывший СССР           51370
Интернет и СМИ        44433
Из жизни              27519
Дом                   21734
Силовые структуры     11223
Ценности               7581
Бизнес                 7375
Путешествия            6370
Name: count, dtype: int64

Возьмем для каждого класса опред. количество samples, чтобы невелировать дисбаланс классов, сохраняя при этом 100_000 объектов.

In [85]:
quotas = {
    "Россия": 9_000, 
    "Мир": 9_000, 
    "Экономика": 9_000, 
    "Спорт": 9_000, 
    "Культура": 9_000, 
    "Бывший СССР": 9_000, 
    "Наука и техника": 9_000, 
    "Интернет и СМИ": 9_000, 
    "Из жизни": 5_000, 
    "Дом": 5_000, 
    "Силовые структуры": 6_000, 
    "Ценности": 4_000, 
    "Бизнес": 4_000, 
    "Путешествия": 4_000
}

balanced_df = pd.concat([df_w[df_w["topic"] == cat].sample(n, random_state=42) for cat, n in quotas.items()])
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_df.shape

(100000, 2)

## 2. Обрабатываем полученные данные
Перед обучением немного преообразуем данные, а именно:

- убераем html-разметку
- приведем к нижнему регистру
- оставляем только кириллицу
- удаляем стоп-слова
- убираем короткие слова (менее 3 символов)

In [7]:
nltk.download("stopwords")
nltk.download("punkt")

russian_stopwords = set(stopwords.words("russian")) 

# def preprocess_text(text, lemmatize=True):
#     soup = BeautifulSoup(text, "html.parser")
#     for data in soup(['style', 'script']):
#         data.decompose()
        
#     text = ' '.join(soup.stripped_strings)
#     text = ' '.join(emoji.replace_emoji(text, replace='').split())
#     text = text.lower()
#     text = re.sub(r'[^а-яё ]', ' ', text)
    
#     tokens = word_tokenize(text)
#     tokens = [word for word in tokens if word not in russian_stopwords]
    
#     if lemmatize:
#         tokens = [mystem.lemmatize(word)[0] for word in tokens]
    
#     return " ".join(tokens)

def preprocess_text(text):
    text = BeautifulSoup(text, "html.parser")
    text = text.lower()    
    text = re.sub(r'[^а-яё\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     /nfs/home/rgurtsiev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /nfs/home/rgurtsiev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Обработаем теперь имеющий датасет.

In [88]:
from tqdm import tqdm
tqdm.pandas()

n_jobs = 10

balanced_df["cleaned_full_text"] = Parallel(n_jobs=n_jobs)(
    delayed(preprocess_text)(text) for text in tqdm(balanced_df["full_text"], desc="Processing Text")
)

Processing Text: 100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:15<00:00, 6391.32it/s]


In [90]:
# balanced_df.to_csv('cleaned-lenta-ru-news-100k.csv', index=False)
# balanced_df = pd.read_csv('cleaned-lenta-ru-news-100k.csv')

In [91]:
balanced_df['topic'].value_counts()

topic
Экономика            9000
Наука и техника      9000
Бывший СССР          9000
Спорт                9000
Культура             9000
Мир                  9000
Россия               9000
Интернет и СМИ       9000
Силовые структуры    6000
Из жизни             5000
Дом                  5000
Бизнес               4000
Ценности             4000
Путешествия          4000
Name: count, dtype: int64

Стало лучше)

In [101]:
X = balanced_df['cleaned_full_text']
y = balanced_df['topic']

label2id = {label:id for id, label in enumerate(y.unique())}
id2label = {id:label for id, label in enumerate(y.unique())}

y = y.map(label2id)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print(f"Размер обучающей выборки: {len(X_train)}")
print(f"Размер валидационной выборки: {len(X_val)}")
print(f"Размер тестовой выборки: {len(X_test)}")

Размер обучающей выборки: 60000
Размер валидационной выборки: 20000
Размер тестовой выборки: 20000


# 3. Базовое качество с DummyClassifier

In [102]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train, y_train)
y_pred = dummy.predict(X_test)

print(f"Dummy Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Dummy Accuracy: 0.0900


# 4. Обучаем модель на основе другого подхода 
- CountVectorizer 
- TfidfVectorizer

В качестве модели будем использовать LogisticRegression

In [105]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipe_count = Pipeline([
    ('vec', CountVectorizer(max_features=20_000)),
    ('clf', LogisticRegression(max_iter=1000))
])
pipe_count.fit(X_train, y_train)
y_pred_count = pipe_count.predict(X_val)


pipe_tfidf = Pipeline([
    ('vec', TfidfVectorizer(max_features=20_000)),
    ('clf', LogisticRegression(max_iter=1000))
])
pipe_tfidf.fit(X_train, y_train)
y_pred_tfidf = pipe_tfidf.predict(X_val)

print(f'CountVectorizer accuracy: {accuracy_score(y_val, y_pred_count):.4f}')
print(f'TfidfVectorizer accuracy: {accuracy_score(y_val, y_pred_tfidf):.4f}')

CountVectorizer accuracy: 0.7999
TfidfVectorizer accuracy: 0.8145


# 5. Попробуйте улучшить качество, подобрав оптимальные гиперпараметры трансформаций и модели на кросс-валидации
Поскольку TF-IDF показал лучший результат, то его мы и будем улучшать)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'vec__ngram_range': [(1, 1), (1, 2)],
    'vec__max_features': [10_000, 20_000, 30_000],
    'clf__C': [0.1, 1, 10]
}

grid = GridSearchCV(pipe_tfidf, params, cv=3, n_jobs=10)
grid.fit(X_val, y_val)

print(f'Best params: {grid.best_params_}')
print(f'Best CV accuracy: {grid.best_score_:.4f}')

In [ ]:
best_model = grid.best_estimator_
y_test_pred = best_model.predict(X_test)

print(f'Test accuracy: {accuracy_score(y_test, y_test_pred):.4f}')